In [ ]:
# testing for multiple files


import pdfplumber
import openpyxl
from openpyxl import Workbook, load_workbook

def extract_multiple_pdfs_to_excel(list_excel_path, output_excel_path):
    """
    Extracts questions and answers from multiple PDFs and writes them to a single Excel file.
    
    Args:
    list_excel_path (str): Path to the Excel file containing a list of PDF paths.
    output_excel_path (str): Path for the output Excel file.
    """
    # Read the Excel file containing the list of PDF paths
    list_workbook = load_workbook(list_excel_path)
    list_sheet = list_workbook.active

    # Create or load an Excel workbook for the output
    output_workbook = Workbook()
    output_sheet = output_workbook.active
    output_sheet.append(["File Name", "Header", "Question", "Answer"])

    # Loop through each row in the PDF list sheet
    for row in list_sheet.iter_rows(values_only=True):
        pdf_path = row[0]  # Assume the first column contains the PDF path
        filename = pdf_path.split('/')[-1]

        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            # Extract the first line of the first page
            first_page = pdf.pages[0]
            first_line = first_page.extract_text().split('\n')[0]

            # Combine text from all pages
            full_text = ''
            for page in pdf.pages:
                full_text += page.extract_text() + '\n'

            # Split the text into sections based on 'Objection' and 'Response'
            sections = full_text.split('Objection')
            for section in sections[1:]:
                question, answer = section.split('Response', 1)
                question = 'Objection' + question.strip()
                answer = answer.strip()
                output_sheet.append([filename, first_line, question, answer])

    # Save the combined workbook
    output_workbook.save(output_excel_path)

# Path to the Excel file containing a list of PDF paths
list_excel_path = r"D:\Scholarship\Objection database code\List.xlsx"
# Path for the combined output Excel file
output_excel_path = r"D:\Scholarship\Objection database code\combined_output.xlsx"

# Extract questions and answers from multiple PDFs and write them to a single Excel file
extract_multiple_pdfs_to_excel(list_excel_path, output_excel_path)
